In [1]:
import os
hml_test_key = os.environ["ARYN_TEST_API_KEY"]

In [2]:
import sycamore
from sycamore.transforms.partition import ArynPartitioner
from sycamore import MaterializeSourceMode

context = sycamore.init()

In [3]:
from aryn_sdk.client.client import Client
ac = Client(aryn_url="https://test-api.aryn.ai", aryn_api_key=hml_test_key)

In [4]:
x = ac.list_docsets()
#x.curr_page
docset_id = "aryn:ds-ry75756y0pbpilq5kojh65q"

In [5]:
read_ds = context.read.aryn(aryn_url="http://localhost:8000/v1/storage", aryn_api_key=hml_test_key, docset_id=docset_id)

/var/folders/79/55b6r6m11vbb45ynfrj8l7tr0000gn/T/ipykernel_74191/648800114.py:1: FutureWarning: Class aryn is experimental and may change in the future.
  read_ds = context.read.aryn(aryn_url="http://localhost:8000/v1/storage", aryn_api_key=hml_test_key, docset_id="aryn:ds-z4wd1ey42j1t4alkity74wj")
/Users/hmlin/Aryn/haystack-workshop-2025/sycamore/lib/sycamore/sycamore/reader.py:662: FutureWarning: Class ArynReader is experimental and may change in the future.
  dr = ArynReader(


In [6]:
#d = read_ds.take(1)[0]

In [7]:
#oe = d.properties.pop("_original_elements")
#d.properties

In [8]:
def drop_orig_elts(d):
    d.properties.pop("_original_elements", None)
    return d

In [9]:
filterds = read_ds.filter(lambda d: "airbnb" in d.properties['name']).map(drop_orig_elts)

In [10]:
# filterds.show()

In [11]:
from sycamore.query.execution.operations import summarize_data
from sycamore.transforms.summarize import MultiStepDocumentSummarizer, EtCetera, OneStepDocumentSummarizer
from sycamore.llms.llms import LLMMode
from sycamore.llms.openai import OpenAI, OpenAIModels
from sycamore.functions.tokenizer import OpenAITokenizer

question = "Who said the most across the AirBnB earnings calls?"
data_desc = "some AirBnB earnings calls"

oai = OpenAI(OpenAIModels.GPT_4O)
oaitk = OpenAITokenizer(OpenAIModels.GPT_4O.value.name, max_tokens=100_000)
summarizer = OneStepDocumentSummarizer(
    llm=oai, 
    #llm_mode=LLMMode.ASYNC, 
    question=question, 
    #data_description=data_desc, 
    fields=[EtCetera],
    tokenizer=oaitk
)
summarize_data(
    llm=oai,
    question=question,
    data_description=data_desc,
    input_data=[filterds],
    docset_summarizer=summarizer
)

/Users/hmlin/Aryn/haystack-workshop-2025/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-26 10:54:20,031	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-26 10:54:20,970	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 
2025-03-26 10:54:21,658	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/Users/hmlin/Aryn/haystack-workshop-2025/sycamore/lib/sycamore/sycamore/connectors/aryn/ArynReader.py:127: FutureWarning: Class ArynClient is experimental and may change in the future.
  client = ArynClient(params.aryn_url, params.api_key)
(pid=74215) INFO:

"In the Airbnb earnings calls, Brian Chesky, the Co-Founder and Chief Executive Officer, is the most frequent speaker. He provides detailed insights into the company's performance, strategic initiatives, and future plans. Chesky discusses various topics, including Airbnb's financial results, growth strategies, and the introduction of new features and services. He also addresses questions from analysts, highlighting the company's focus on expanding its core business, improving service quality, and exploring new markets. Chesky's role as the primary spokesperson underscores his leadership in communicating Airbnb's vision and progress to investors and stakeholders."

In [47]:
from aryn_sdk.types.search import SearchRequest

def print_element_search_results(sr):
    results = sr.value.results
    for elt in results:
        print(f"{elt['properties']['speaker_name']} -- {elt['properties']['speaker_role']}")
        print(f"    {elt['text_representation']}")
        print("-" * 80)

sr = ac.search(
    docset_id=docset_id, 
    query=SearchRequest(
        query="What was the change in stock price on the day of the Q2 2024 AirBnB earnings call?",
        query_type="lexical",
        properties_filter="(properties.entity.earnings_call.company_ticker <> \"ABNB\")",
        return_type="doc",
    )
)
sr

In [48]:
def print_doc_sr(sr):
    results = sr.value.results
    for doc in results:
        print(f"{doc['properties']['entity']['earnings_call']}")
        print('=' * 80)

print_doc_sr(sr)

{'company_name': 'Airbnb', 'company_ticker': 'ABNB', 'quarter': 'Q4', 'date': 'Feb 13, 2025', 'stock_price': 122.86}
{'company_name': 'Airbnb', 'company_ticker': 'ABNB', 'quarter': 'Q2', 'date': 'Aug 06, 2024', 'stock_price': 122.86}
{'company_name': 'Airbnb', 'company_ticker': 'ABNB', 'quarter': 'Q1', 'date': 'May 08, 2024', 'stock_price': 129.35}
{'company_name': 'Airbnb', 'company_ticker': 'ABNB', 'quarter': 'Q3', 'date': 'Nov 07, 2024', 'stock_price': 129.31}
{'company_name': 'Datadog', 'company_ticker': 'DDOG', 'quarter': 'Q2', 'date': 'Aug 08, 2024', 'stock_price': 101.8}
{'company_name': 'Tesla', 'company_ticker': 'TSLA', 'quarter': 'Q4', 'date': 'Jan 29, 2025', 'stock_price': 249.98}
{'company_name': 'Palantir Technologies', 'company_ticker': 'PLTR', 'quarter': 'Q2', 'date': 'Aug 05, 2024', 'stock_price': 86.24}
{'company_name': 'Cisco Systems', 'company_ticker': 'CSCO', 'quarter': 'Q4', 'date': 'Aug 14, 2024', 'stock_price': 60.5}
{'company_name': 'Atlassian', 'company_ticker'

In [49]:
# sr.value.results

In [41]:
print_element_search_results(sr)